# El precio justo

Hoy crearemos otra pieza del rompecabezas: un ScanningAgent que busca ofertas prometedoras suscribiéndose a feeds RSS.

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [ ]:
# Inicialización y constantes

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [ ]:
deals = ScrapedDeal.fetch(show_progress=True)

In [ ]:
len(deals)

In [ ]:
deals[44].describe()

In [ ]:
system_prompt = """Identifica y resume las 5 ofertas más detalladas de una lista, seleccionando las ofertas que tengan la descripción más detallada y de alta calidad y el precio más claro.
Responde estrictamente en JSON sin explicación, usando este formato. Debes proporcionar el precio como un número derivado de la descripción. Si el precio de una oferta no está claro, no incluyas esa oferta en tu respuesta.
Lo más importante es que respondas con las 5 ofertas que tengan la descripción del producto más detallada con el precio. No es importante mencionar los términos de la oferta; lo más importante es una descripción detallada del producto.
Ten cuidado con los productos que se describen como "$XXX de descuento" o "reducidos en $XXX"; este no es el precio real del producto. Solo responde con productos cuando estés muy seguro del precio.

{"deals": [
    {
        "product_description": "Un resumen claro del producto en 4 o 5 oraciones. Los detalles del artículo son mucho más importantes que el motivo por el que es una buena oferta. Evita mencionar descuentos y cupones; céntrese en el artículo en sí. Debe haber un párrafo de texto para cada artículo que elija.",
        "price": 99.99,
        "url": "la url donde se halla el artículo"
    },
    ...
]}"""

In [ ]:
user_prompt = """Responde con las 5 ofertas más prometedoras de esta lista, seleccionando aquellas que tengan la descripción del producto más detallada y de mayor calidad y un precio claro.
Responde estrictamente en JSON, y solo en JSON. Debes reformular la descripción para que sea un resumen del producto en sí, no los términos de la oferta.
Recuerda responder con un párrafo de texto en el campo product_description para cada uno de los 5 artículos que selecciones.
Ten cuidado con los productos que se describen como "$XXX de descuento" o "reducidos en $XXX": este no es el precio real del producto. Solo responde con productos cuando estés muy seguro del precio.

Ofertas:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [ ]:
print(user_prompt[:2000])

In [ ]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [ ]:
result = get_recommendations()

In [ ]:
len(result.deals)

In [ ]:
result.deals[1]

In [ ]:
from agents.scanner_agent import ScannerAgent

In [ ]:
agent = ScannerAgent()
result = agent.scan()

In [ ]:
result